# 1 BPR Algorithm
1.1 Personalized Ranking from Implicit Feedback
The task of personalized ranking aims at providing each user a ranked list of items (recommendations). This is very common in scenarios where recommender systems are based on implicit user behavior (e.g. purchases, clicks). The available observations are only positive feedback where the non-observed ones are a mixture of real negative feedback and missing values.

One usual approach for item recommendation is directly predicting a preference score $\hat{x}_{u,i}$ given to item $i$ by user $u$. BPR uses a different approach by using item pairs $(i, j)$ and optimizing for the correct ranking given preference of user $u$, thus, there are notions of positive and negative items. The training data $D_S : U \times I \times I$ is defined as:

$$D_S = \{(u, i, j) \mid i \in I^{+}_{u} \wedge j \in I \setminus I^{+}_{u}\}$$
where user $u$ is assumed to prefer $i$ over $j$ (i.e. $i$ is a positive item and $j$ is a negative item).
1.2 Objective Function
From the Bayesian perspective, BPR maximizes the posterior probability over the model parameters $\Theta$ by optimizing the likelihood function $p(i &gt;_{u} j | \Theta)$ and the prior probability $p(\Theta)$.

$$p(\Theta \mid &gt;_{u}) \propto p(i &gt;_{u} j \mid \Theta) \times p(\Theta)$$
The joint probability of the likelihood over all users $u \in U$ can be simplified to:

$$ \prod_{u \in U} p(&gt;_{u} \mid \Theta) = \prod_{(u, i, j) \in D_S} p(i &gt;_{u} j \mid \Theta) $$
The individual probability that a user $u$ prefers item $i$ to item $j$ can be defined as:

$$ p(i &gt;_{u} j \mid \Theta) = \sigma (\hat{x}_{uij}(\Theta)) $$
where $\sigma$ is the logistic sigmoid:

$$ \sigma(x) = \frac{1}{1 + e^{-x}} $$
The preference scoring function $\hat{x}_{uij}(\Theta)$ could be an arbitrary real-valued function of the model parameter $\Theta$. Thus, it makes BPR a general framework for modeling the relationship between triplets $(u, i, j)$ where different model classes like matrix factorization could be used for estimating $\hat{x}_{uij}(\Theta)$.

For the prior, one of the common pratices is to choose $p(\Theta)$ following a normal distribution, which results in a nice form of L2 regularization in the final log-form of the objective function.

$$ p(\Theta) \sim N(0, \Sigma_{\Theta}) $$
To reduce the complexity of the model, all parameters $\Theta$ are assumed to be independent and having the same variance, which gives a simpler form of the co-variance matrix $\Sigma_{\Theta} = \lambda_{\Theta}I$. Thus, there are less number of hyperparameters to be determined.

The final objective of the maximum posterior estimator:

$$ J = \sum_{(u, i, j) \in D_S} \text{ln } \sigma(\hat{x}_{uij}) - \lambda_{\Theta} ||\Theta||^2 $$
where $\lambda_\Theta$ are the model specific regularization paramerters.

1.3 Learning with Matrix Factorization
Stochastic Gradient Descent
As the defined objective function is differentible, gradient descent based method for optimization is naturally adopted. The gradient of the objective $J$ with respect to the model parameters:

$$
\begin{align}
\frac{\partial J}{\partial \Theta} &amp; = \sum_{(u, i, j) \in D_S} \frac{\partial}{\partial \Theta} \text{ln} \ \sigma(\hat{x}_{uij}) - \lambda_{\Theta} \frac{\partial}{\partial \Theta} ||\Theta||^2 \\
&amp; \propto \sum_{(u, i, j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot  \frac{\partial}{\partial \Theta} \hat{x}_{uij} - \lambda_{\Theta} \Theta
\end{align}
$$
Due to slow convergence of full gradient descent, we prefer using stochastic gradient descent to optimize the BPR model. For each triplet $(u, i, j) \in D_S$, the update rule for the parameters:

$$ \Theta \leftarrow \Theta + \alpha \Big( \frac{e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot \frac{\partial}{\partial \Theta} \hat{x}_{uij} + \lambda_\Theta \Theta \Big) $$
Matrix Factorization for Preference Approximation
As mentioned earlier, the preference scoring function $\hat{x}_{uij}(\Theta)$ could be approximated by any real-valued function. First, the estimator $\hat{x}_{uij}$ is decomposed into:

$$ \hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} $$
The problem of estimating $\hat{x}_{ui}$ is a standard collaborative filtering formulation, where matrix factorization approach has shown to be very effective. The prediction formula can written as dot product between user feature vector $w_u$ and item feature vector $h_i$:

$$ \hat{x}_{ui} = \langle w_u , h_i \rangle = \sum_{f=1}^{k} w_{uf} \cdot h_{if} $$
The derivatives of matrix factorization with respect to the model parameters are:

$$
\frac{\partial}{\partial \theta} \hat{x}_{uij} = 
\begin{cases}
    (h_{if} - h_{jf})  &amp; \text{if } \theta = w_{uf} \\
    w_{uf}             &amp; \text{if } \theta = h_{if} \\
    -w_{uf}            &amp; \text{if } \theta = h_{jf} \\
    0                  &amp; \text{else}
\end{cases}
$$
In theory, any kernel can be used to estimate $\hat{x}_{ui}$ besides the dot product $ \langle \cdot , \cdot \rangle $. For example, k-Nearest-Neighbor (kNN) has also been shown to achieve good performance.

Analogies to AUC optimization
By optimizing the objective function of BPR model, we effectively maximizing AUC measurement. To keep the notebook focused, please refer to the paper for details of the analysis (Section 4.1.1).

# 2 Cornac implementation of BPR
BPR is implemented in the Cornac framework as part of the model collections.

Detailed documentations of the BPR model in Cornac can be found here.
Source codes of the BPR implementation is available on the Cornac Github repository, which can be found here.

# 3 Cornac BPR game recommender
### 3.1 Load and split data
To evaluate the performance of item recommendation, we adopted the provided python_random_split tool for the consistency. Data is randomly split into training and test sets with the ratio of 75/25.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

sns.set_style("white")

In [32]:
df = pd.read_csv('steam-200k-cleaned.csv')
df

,user,game,hours,purchase,play
0,5250,Alien Swarm,4.9,1,1
1,5250,Cities Skylines,144.0,1,1
2,5250,Counter-Strike,0.0,1,0
3,5250,Counter-Strike Source,0.0,1,0
4,5250,Day of Defeat,0.0,1,0
...,...,...,...,...,...
128799,309626088,Age of Empires II HD Edition,6.7,1,1
128800,309812026,Counter-Strike Nexon Zombies,0.0,1,0
128801,309812026,Robocraft,0.0,1,0
128802,309824202,Dota 2,0.7,1,1


In [33]:
# Drop NaN columns
data = df.dropna()
data = df.copy()

In [36]:
# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['game'] = data['game'].astype("category")
data['user_id'] = data['user'].cat.codes
data['game_id'] = data['game'].cat.codes

data['user_id'] = data['user_id'].astype(np.int64)
data['game_id'] = data['game_id'].astype(np.int64)

data = data.drop(['user', 'game','purchase','play'], axis=1)

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128804 entries, 0 to 128803
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   hours    128804 non-null  float64
 1   user_id  128804 non-null  int64  
 2   game_id  128804 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 2.9 MB


In [38]:
import sys
sys.path.append("../../")
import os
import cornac
# import papermill as pm
import pandas as pd
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.cornac.cornac_utils import predict_ranking
from reco_utils.common.timer import Timer
from reco_utils.common.constants import SEED

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
Cornac version: 1.8.0


In [39]:
# top k items to recommend
TOP_K = 10

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

In [40]:
train, test = python_random_split(data, 0.75)

### 3.2 Cornac Dataset
To work with models implemented in Cornac, we need to construct an object from Dataset class.

Dataset Class in Cornac serves as the main object that the models will interact with. In addition to data transformations, Dataset provides a bunch of useful iterators for looping through the data, as well as supporting different negative sampling techniques.

In [41]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=123)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 1439
Number of items: 10900


C:\Users\HP\anaconda3\envs\metis\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 45784 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


### 3.3 Train the BPR model
The BPR has a few important parameters that we need to consider:

k: controls the dimension of the latent space (i.e. the size of the vectors $w_u$ and $h_i$ ).
max_iter: defines the number of iterations of the SGD procedure.
learning_rate: controls the step size $\alpha$ in the gradient update rules.
lambda_reg: controls the L2-Regularization $\lambda$ in the objective function.
Note that different values of k and max_iter will affect the training time.

We will here set k to 200, max_iter to 100, learning_rate to 0.01, and lambda_reg to 0.001. To train the model, we simply need to call the fit() method.

In [64]:
# Model parameters
NUM_FACTORS = 40
NUM_EPOCHS = 20

bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=123
)

In [65]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))


Optimization finished!
Took 0.8659 seconds for training.


### 3.4 Prediction and Evaluation
Now that our model is trained, we can produce the ranked lists for recommendation. Every recommender models in Cornac provide rate() and rank() methods for predicting item rated value as well as item ranked list for a given user. To make use of the current evaluation schemes, we will through predict() and predict_ranking() functions inside cornac_utils to produce the predictions.

Note that BPR model is effectively designed for item ranking. Hence, we only measure the performance using ranking metrics.

In [66]:
import pandas as pd
import numpy as np

from reco_utils.common.constants import (
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_PREDICTION_COL)

In [67]:
def predict_ranking(
    model,
    data,
    usercol=DEFAULT_USER_COL,
    itemcol=DEFAULT_ITEM_COL,
    predcol=DEFAULT_PREDICTION_COL,
    remove_seen=False,
):
    """Computes predictions of recommender model from Cornac on all users and items in data.
    It can be used for computing ranking metrics like NDCG.
    Args:
        model (cornac.models.Recommender): a recommender model from Cornac
        data (pd.DataFrame): the data from which to get the users and items
        usercol (str): name of the user column
        itemcol (str): name of the item column
        remove_seen (bool): flag to remove (user, item) pairs seen in the training data
    Returns:
        pd.DataFrame: dataframe with usercol, itemcol, predcol
    """
    users, items, preds = [], [], []
    item = list(model.train_set.iid_map.keys())
    
    for uid, user_idx in model.train_set.uid_map.items():
        user = [uid] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(model.score(user_idx).tolist())

    all_predictions = pd.DataFrame(
        data={usercol: users, itemcol: items, predcol: preds}
    )

    if remove_seen:
        tempdf = pd.concat(
            [
                data[[usercol, itemcol]],
                pd.DataFrame(
                    data=np.ones(data.shape[0]), columns=["dummycol"], index=data.index
                ),
            ],
            axis=1,
        )
        print (tempdf)
        print (all_predictions)
        merged = pd.concat([tempdf, all_predictions], keys=['usercol', 'itemcol'], join="outer")
        
        return merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
    
    else:
        
        return all_predictions

In [68]:
with Timer() as t:
     all_predictions = predict_ranking(
                            bpr,
                            train,
                            usercol='user_id',
                            itemcol='game_id',
                            remove_seen=True,
                        )  
print("Took {} seconds for prediction.".format(t))

        user_id  game_id  dummycol
51123      1830     4039       1.0
51963      1869      313       1.0
64132      2580     4257       1.0
20623       665     3060       1.0
13649       395     4313       1.0
...         ...      ...       ...
128106    11994      981       1.0
103694     6406     4257       1.0
860          25      492       1.0
15795       473      629       1.0
121958    10183      452       1.0

[96603 rows x 3 columns]
          user_id  game_id  prediction
0             0.0     1830    0.395350
1             0.0     1869    2.074868
2             0.0     2580    0.376701
3             0.0      665    2.320160
4             0.0      395    2.200058
...           ...      ...         ...
15685095    373.0     4167   -0.226598
15685096    373.0    12064   -0.239423
15685097    373.0     7610   -0.250838
15685098    373.0     9550   -0.293032
15685099    373.0    11994   -0.270246

[15685100 rows x 3 columns]
Took 27.9403 seconds for prediction.


In [69]:
all_predictions.head()

user_id  game_id  prediction
itemcol 0      0.0     1830    0.395350
        1      0.0     1869    2.074868
        2      0.0     2580    0.376701
        3      0.0      665    2.320160
        4      0.0      395    2.200058

In [70]:
all_predictions.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15685100 entries, ('itemcol', 0) to ('itemcol', 15685099)
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     float64
 1   game_id     int64  
 2   prediction  float64
dtypes: float64(2), int64(1)
memory usage: 1.2+ GB


In [71]:
all_predictions['game_id'] = all_predictions['game_id'].astype(np.int64)
all_predictions['user_id'] = all_predictions['user_id'].astype(np.int64)
all_predictions.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15685100 entries, ('itemcol', 0) to ('itemcol', 15685099)
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     int64  
 1   game_id     int64  
 2   prediction  float64
dtypes: float64(1), int64(2)
memory usage: 1.2+ GB


In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32201 entries, 31140 to 117047
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   hours    32201 non-null  float64
 1   user_id  32201 non-null  int64  
 2   game_id  32201 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 1006.3 KB


In [74]:
k = 10
eval_map = map_at_k(test, all_predictions, 
                    col_user='user_id',
                    col_item='game_id', 
                    col_rating='hours', 
                    col_prediction='prediction', 
                    k=k)

eval_ndcg = ndcg_at_k(test, all_predictions, col_user='user_id',
                    col_item='game_id', 
                    col_rating='hours', col_prediction='prediction', k=k)


eval_precision = precision_at_k(test, all_predictions, col_user='user_id',
                    col_item='game_id', 
                    col_rating='hours', col_prediction='prediction', k=k)


eval_recall = recall_at_k(test, all_predictions, col_user='user_id',
                    col_item='game_id', 
                    col_rating='hours', col_prediction='prediction', k=k)


print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.000168
NDCG:	0.001418
Precision@K:	0.001075
Recall@K:	0.000352
